In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from tqdm import tqdm
import pickle as pk
# data read
author_emd = {}
dataset_emd = {}
method_emd = {}


In [28]:
fAuthor = open(
    "../../data/HetGNNdata/0423author_node_embedding.txt", "r")
for line in tqdm(fAuthor):
    line = line.strip().split(' ')
    temp_ebd = list(map(float, line[1:]))
    author_emd[line[0]] = np.array(temp_ebd)
fAuthor.close()

fDataset = open(
    "../../data/HetGNNdata/0423dataset_node_embedding.txt", "r")
for line in tqdm(fDataset):
    line = line.strip().split(' ')
    temp_ebd = list(map(float, line[1:]))
    dataset_emd[line[0]] = np.array(temp_ebd)
fDataset.close()

fMethod = open(
    "../../data/HetGNNdata/0423method_node_embedding.txt", "r")
for line in tqdm(fMethod):
    line = line.strip().split(' ')
    temp_ebd = list(map(float, line[1:]))
    method_emd[line[0]] = np.array(temp_ebd)
fMethod.close()


522614it [00:17, 29355.96it/s]
244it [00:00, 27117.39it/s]
307it [00:00, 27913.53it/s]


In [16]:
authorBreastSet = set(author_emd.keys())
dataBreastSet = set(dataset_emd.keys())
methodBreastSet = set(method_emd.keys())

In [4]:
# add author name!
f = open("../../data/HetGNNdata/Authors.csv", "r", encoding='utf-8')
author_name = {}
author_CareerAge = {}
author_PaperNum = {}
author_CitedNum = {}
author_mainAffiliation = {}
for line in tqdm(f):
    line = line.strip().split(',')
    if line[0] in authorBreastSet:
        # author_name[int(line[0])] = line[4]
        # author_CareerAge[int(line[0])] = line[1]
        # author_PaperNum[int(line[0])] = line[2]
        author_CitedNum[line[0]] = int(line[3])
        # author_mainAffiliation[int(line[0])] = str(line[5:]).strip(
        #     "\"[\]'").replace('\'', '').replace('\"', '').replace('  ', ' ')
f.close()


8699085it [00:08, 999388.01it/s] 


In [22]:
count = 0
for k,v in author_CitedNum.items():
    if v > 233:
        count += 1
# 49030
print(count)
filter_author_embeddings = {}

for k,v in author_CitedNum.items():
    if v > 233:
        filter_author_embeddings[k] = author_emd[k]

49030


In [23]:
author_emd = filter_author_embeddings

In [24]:
authorBreastSet = set(author_emd.keys())
dataBreastSet = set(dataset_emd.keys())
methodBreastSet = set(method_emd.keys())

In [60]:
len(methodBreastSet)


307

In [8]:
# combine them
dataset_author_method_keys = list(
    dataset_emd.keys()) + list(author_emd.keys()) + list(method_emd.keys())
dataset_author_method_list = list(dataset_emd.values(
)) + list(author_emd.values()) + list(method_emd.values())
dataset_author_method_array = np.array(dataset_author_method_list)
authorBreastSet = set(author_emd.keys())
dataBreastSet = set(dataset_emd.keys())
methodBreastSet = set(method_emd.keys())
# kmeans
dataset_author_method_Kmeans = KMeans(
    n_clusters=5, random_state=0,verbose=1).fit(dataset_author_method_array)
print('kmeans over')

clusters = dataset_author_method_Kmeans.predict(dataset_author_method_array)

dataset_author_method_kmeans_label = {}
for i in range(len(dataset_author_method_keys)):
    dataset_author_method_kmeans_label[dataset_author_method_keys[i]] = clusters[i]

dataset_cluster = {}
author_cluster = {}
method_cluster = {}
for i in dataset_emd.keys():
    dataset_cluster[i] = dataset_author_method_kmeans_label[i]
for i in author_emd.keys():
    author_cluster[int(i)] = dataset_author_method_kmeans_label[i]
for i in method_emd.keys():
    method_cluster[i] = dataset_author_method_kmeans_label[i]

Initialization complete
Iteration 0, inertia 351671.47109569644
Iteration 1, inertia 210612.04257037895
Iteration 2, inertia 208302.9483099296
Iteration 3, inertia 207147.08064308247
Iteration 4, inertia 206736.46323484066
Iteration 5, inertia 206640.2972596178
Iteration 6, inertia 206590.39311607738
Iteration 7, inertia 206563.64728159757
Iteration 8, inertia 206549.07461415604
Iteration 9, inertia 206540.94829197144
Iteration 10, inertia 206535.40249717483
Iteration 11, inertia 206531.3862610431
Iteration 12, inertia 206528.58335282403
Iteration 13, inertia 206527.16642928775
Iteration 14, inertia 206526.2883021576
Iteration 15, inertia 206525.52336545291
Iteration 16, inertia 206524.99912247903
Iteration 17, inertia 206524.6232287938
Iteration 18, inertia 206524.34229593887
Iteration 19, inertia 206524.21580743464
Iteration 20, inertia 206524.14249004197
Iteration 21, inertia 206524.09186071117
Converged at iteration 21: center shift 1.7370653582546325e-06 within tolerance 4.0184707

In [9]:
df_author = pd.DataFrame(pd.Series(author_cluster),columns=['clusterID'])

In [11]:
# df_author.to_csv('../../data/HetGNNdata/authors4atlas.csv', encoding='utf-8')

In [9]:
# 需要：author基于bioentity的类别标签。
'''
Scientists Cluster 0 - Biomarker Discovery
Scientists Cluster 1 - Targeted Therapy
Scientists Cluster 2 - Risk Factors
Scientists Cluster 3 - Genetic Variants
Scientists Cluster 4 - Diagnosis and Treatment
'''
cluster2label = {
    0:'Scientists Cluster 0 - Biomarker Discovery',
    1: 'Scientists Cluster 1 - Targeted Therapy',
    2: 'Scientists Cluster 2 - Risk Factors',
    3:  'Scientists Cluster 3 - Genetic Variants',
    4: 'Scientists Cluster 4 - Diagnosis and Treatment'
}

In [17]:
# add author B2AI!
f = open("../../data/HetGNNdata/B2AiAuthors.csv", "r", encoding = 'utf-8' )
author_B2AI = {}

for line in tqdm(f):
    line = line.strip().split(',')
    if line[0] in authorBreastSet and line[1] != '':
        author_B2AI[int(line[0])] = 1
f.close()
len(author_B2AI)

470584it [00:00, 908811.96it/s]


27

In [18]:
# add dataset name!
f = open("../../data/Dateset_Extraction_20230411.csv", "r")
datasetIDName = {}
for line in tqdm(f):
    line = line.strip().split(',')
    if line[3] in dataset_cluster:
        datasetIDName[line[3]] = line[2]
f.close()
len(datasetIDName)


8448it [00:00, 1207973.28it/s]


244

In [19]:
# add obi name!
f = open("../../data/OBI_Extraction.csv", "r")
obiIDName = {}
for line in tqdm(f):
    line = line.strip().split(',')
    if line[3] in method_cluster:
        obiIDName[line[3]] = line[2]
f.close()

64048it [00:00, 1173645.02it/s]


In [40]:
# add author name!
f = open("../../data/HetGNNdata/Authors.csv", "r", encoding='utf-8')
author_name = {}
author_CareerAge = {}
author_PaperNum = {}
author_CitedNum = {}
author_mainAffiliation = {}
for line in tqdm(f):
    line = line.strip().split(',')
    try:
        int(line[0])
    except:
        continue
    if int(line[0]) in author2ebd4atlas50000:
        author_name[int(line[0])] = line[4]
        # author_CareerAge[int(line[0])] = line[1]
        # author_PaperNum[int(line[0])] = line[2]
        # author_CitedNum[int(line[0])] = int(line[3])
        # author_mainAffiliation[int(line[0])] = str(line[5:]).strip(
        #     "\"[\]'").replace('\'', '').replace('\"', '').replace('  ', ' ')
f.close()
len(author_name)

8699085it [00:16, 512753.98it/s]


49046

In [41]:
len(author2ebd4atlas50000)


49046

In [ ]:
author_name

In [ ]:
author_B2AI

In [25]:
authorPlus2AI_index =list(set(list(map(int,author_emd.keys()))+list(author_B2AI.keys())))
len(authorPlus2AI_index)


49046

In [26]:
author2label = {}

In [27]:
for author in authorPlus2AI_index:
    if author in author_B2AI:
        author2label[author]= 'Bridge2AI Talents'
    else:
        author2label[author]= cluster2label[author_cluster[author]]

In [29]:
author2ebd4atlas50000 = {}
for k,v in author_emd.items():
    if int(k) in author2label:
        author2ebd4atlas50000[int(k)] = v

In [31]:
len(author2ebd4atlas50000)

49046

In [30]:
len(authorPlus2AI_index)


49046

In [43]:
myEmbeddings = list(author2ebd4atlas50000.values()) + list(method_emd.values()) + list(dataset_emd.values())
myEmbeddings=np.array(myEmbeddings)

In [44]:
obiIDName['OBI:0000829'] = 'Sysmex Corporation'

In [21]:
import pandas as pd
import pickle as pk
article_title = pd.read_csv(
    '../../data/breast_cancaer_papers_PKG23.csv')
paper_titleAbstract = {}
article_title['TitleAbstract'] = article_title['ArticleTitle'] + \
    article_title['Abstract']
article_title.dropna(subset=['TitleAbstract'], inplace=True)
# article_title['TitleAbstract'] = article_title['TitleAbstract'].apply(clean_text)
for index, row in article_title.iterrows():
    paper_titleAbstract[str(row['PMID'])] = row['TitleAbstract']

author_paper = pk.load(
    open('../../data/HetGNNdata/author_paper.pkl', 'rb'))

authors_text = {}
for k in author2label.keys():
    authors_text[k] = ''
for k, v in tqdm(author_paper.items()):
    # 构建词组
    if int(k) in author2label:
            # 对应的paper集合
        paper_set = v
        for paper in paper_set:
            try:
                authors_text[int(k)] += ' '+paper_titleAbstract[paper]
            except:
                continue

100%|██████████| 524117/524117 [00:03<00:00, 141861.75it/s]


In [45]:
data = [{'Category': author2label[k], 'Name': ('AID: '+str(k)+'\n'+'Name: '+author_name[k])[0:36]}
        for k in author2ebd4atlas50000.keys() ] + [{'Category': 'Methods', 'Name': (k+obiIDName[k])[0:36]}
        for k in method_cluster.keys() ] + [{'Category': 'Datasets', 'Name': (k+datasetIDName[k])[0:36]}
        for k in dataset_cluster.keys() ]

In [48]:
len(data)

49597

In [77]:
len(author_name)

11

In [92]:
len(myEmbeddings)


562

In [49]:
pk.dump(myEmbeddings, open(
    '../../data/HetGNNdata/myEmbeddings4atlas.pkl', 'wb'))
pk.dump(data, open(
    '../../data/HetGNNdata/data4atlas.pkl', 'wb'))


In [51]:
from nomic import atlas

import nomic
nomic.login('Ot6iTWIge-5zC-si1QLxHFcRtnCT2a2ZS9FmsWz7U4EhH')
import numpy as np
# num_embeddings = 60000
# embeddings = np.random.rand(num_embeddings, 128)

# categories = ['rhizome', 'cartography', 'lindenstrauss']
# data = [{'category': categories[i % len(categories)], 'id': i}
#         for i in range(len(embeddings))]

project = atlas.map_embeddings(embeddings=myEmbeddings,
                               data=data,
                               id_field='Name',
                               colorable_fields=['Category'],
                            #    topic_label_field='Label'
                               )

2023-04-24 05:14:33.008 | INFO     | nomic.project:_create_project:965 - Creating project `tacky-sight` in organization `vinowei21`
2023-04-24 05:14:35.733 | INFO     | nomic.atlas:map_embeddings:100 - Uploading embeddings to Atlas.
10it [00:30,  3.06s/it]
2023-04-24 05:15:06.397 | INFO     | nomic.project:_add_data:1577 - Upload succeeded.
2023-04-24 05:15:06.398 | INFO     | nomic.atlas:map_embeddings:119 - Embedding upload succeeded.
2023-04-24 05:15:10.686 | INFO     | nomic.project:create_index:1282 - Created map `tacky-sight` in project `tacky-sight`: https://atlas.nomic.ai/map/d8e42abe-7fed-4064-8233-03e3bc26336f/85c658b3-c540-4ec7-8885-af141322e4c9
2023-04-24 05:15:10.687 | INFO     | nomic.atlas:map_embeddings:132 - tacky-sight: https://atlas.nomic.ai/map/d8e42abe-7fed-4064-8233-03e3bc26336f/85c658b3-c540-4ec7-8885-af141322e4c9


In [44]:
len(data)

49581